In [1]:
import os
import re
import pickle
from typing import List, Dict, Any, Tuple, Optional
from pathlib import Path
from dataclasses import dataclass

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [3]:
# 1. 기본 사용
from code_dev.optimal_rag_pipeline import OptimalRAGPipeline

pipeline = OptimalRAGPipeline("data/dev_center_guide_allmd_touched.md").setup()
result = pipeline.search("PNS 메시지의 purchaseState 값은 무엇이 있나요?")

docs = result['retrieved_docs']
for doc in docs:
  print(doc.metadata)
  print(doc.page_content)
  print("-" * 100)

print(f"검색된 문서 수: {len(docs)}")
print("=" * 100)
print(f"성공률: {result['performance']['relevance_score']*100:.1f}%")



🚀 최적 RAG 파이프라인 설정 시작
🚀 최적 문서 분할 시작...
📊 계층별 분할 결과:
  major: 44개 원본 문서
  medium: 83개 원본 문서
  minor: 368개 원본 문서
  major 최종: 47개 청크
  medium 최종: 85개 청크
  minor 최종: 785개 청크
✅ 총 917개 최적화된 문서 생성

📊 문서 품질 검증:
  PNS 관련: 36개 (3.9%)
  purchaseState 포함: 27개 (2.9%)
  PNS+purchaseState: 4개 (0.4%)
⚠️  PNS+purchaseState 문서가 부족할 수 있습니다.
🔧 최적 Retriever 초기화 - 917개 문서
🔧 HybridScoring 검색기 구축 중...
✅ 최적 검색기 구축 완료
✅ 최적 RAG 파이프라인 설정 완료!
{'Header 1': '07. PNS(Payment Notification Service) 이용하기', 'Header 2': '**PNS 상세**', 'Header 3': 'PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)', 'Header 4': '**Signature 검증 방법**', 'chunk_index': 3, 'hierarchy_level': 'minor', 'title_hierarchy': '07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버) > **Signature 검증 방법**', 'source_strategy': 'optimal_minor', 'chunk_size': 780, 'contains_pns': True, 'contains_purchasestate': True, 'pns_purchasestate_both': True, 'content_quality_score': 0.53, 'keyword_density': 0.16666

In [ ]:
# 2. 프로덕션 사용
from code_dev.production_rag_config import ProductionRAGSystem, create_production_config

config = create_production_config()
rag_system = ProductionRAGSystem(config)
rag_system.initialize("your_document.md")
result = rag_system.search("질문")